In [1]:
import json
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)


In [2]:
doc=[]

for i in docs_raw:
    #print(i)
    for j in i['documents']:
        #print(j)
        doc.append(j)


In [3]:
doc[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?'}

In [4]:
!pip install -U sentence-transformers

https://huggingface.co/sentence-transformers/all-mpnet-base-v2

In [5]:
from sentence_transformers import SentenceTransformer

model=SentenceTransformer("all-mpnet-base-v2")

/usr/local/python/3.12.1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
sentences = ["This is an example sentence"]
embeddings = model.encode(sentences)
print("Length of embeddings",len(embeddings[0]))
print(embeddings)

Length of embeddings 768
[[ 2.25026198e-02 -7.82917812e-02 -2.30307057e-02 -5.10002021e-03
  -8.03404152e-02  3.91321853e-02  1.13428487e-02  3.46482755e-03
  -2.94573940e-02 -1.88930500e-02  9.47433710e-02  2.92748362e-02
   3.94859686e-02 -4.63165864e-02  2.54246108e-02 -3.21999937e-02
   6.21928647e-02  1.55591555e-02 -4.67795320e-02  5.03901206e-02
   1.46113681e-02  2.31413674e-02  1.22066317e-02  2.50695869e-02
   2.93651992e-03 -4.19821627e-02 -4.01032949e-03 -2.27844268e-02
  -7.68596493e-03 -3.31090875e-02  3.22119035e-02 -2.09992491e-02
   1.16731236e-02 -9.85073820e-02  1.77932645e-06 -2.29932070e-02
  -1.31140957e-02 -2.80222688e-02 -6.99969903e-02  2.59314068e-02
  -2.89501939e-02  8.76336098e-02 -1.20919058e-02  3.98604944e-02
  -3.31381559e-02  3.59108113e-02  3.46099101e-02  6.49783984e-02
  -3.00817620e-02  6.98188320e-02 -3.99514567e-03 -1.01597363e-03
  -3.50184999e-02 -4.36567403e-02  5.08025698e-02  4.68757562e-02
   5.39663769e-02 -4.03008610e-02  3.20141274e-03  

In [7]:
doc[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?'}

In [12]:
o=[]
for p in doc:
    p['text_vector']=model.encode(p['text']).tolist()
    o.append(p)
    break

In [13]:
o[1]

IndexError: list index out of range

https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/getting-started-python.html

In [ ]:
!pip install elasticsearch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:

from elasticsearch import Elasticsearch
client = Elasticsearch(
    'https://fictional-sniffle-q46qxwqvv7q34gvj-9200.app.github.dev/'
)

client.info()

ObjectApiResponse({'name': '3518c00a61bf', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Dq8zeXmsT1mkdPaXkYdqyA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [21]:
index_name = "course-questions"

client.indices.delete(index=index_name, ignore_unavailable=True)
client.indices.create(index=index_name, body=index_settings)
index_name = "course-questions"


In [24]:
	
for doc in o:
    try:
        client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [26]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [27]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [29]:
res = client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'TOCteJEBiagFpT0cC-rL',
  '_score': 0.5521814,
  '_source': {'question': 'Course - When will the course start?',
   'section': 'General course-related questions',
   'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."}}]

In [30]:

knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [32]:
response = client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [34]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'TOCteJEBiagFpT0cC-rL',
  '_score': 1.7029097,
  '_source': {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
   'section': 'General course-related questions',
   'question': 'Course - When will the course start?',
   'text_vector': [-0.03570346161723137,
    -0.06891428679227829,
    -0.04448342323303223,
    0.006250153295695782,
    -0.042247094213962555,
    -0.00645399559289217,
    0.02754814364016056,
    -0.01861027255654335,
    -0.05203750357031822,
    -0.0024047393817454576,
    0.05745450034737587,
    -0.01559260